In [1]:
# %cd yolo_cam
%cd C:\Users\G.oneH\Multicampus\FinalProject\yolo_cam

C:\Users\G.oneH\Multicampus\FinalProject\yolo_cam


In [2]:
import cv2
import numpy as np
import pandas as pd

import os

from tensorflow.keras.preprocessing import image

In [3]:
net = cv2.dnn.readNet("yolov4-obj_1000.weights", "yolov4-obj.cfg")

In [4]:
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [5]:
url = "http://192.168.0.4:8080/video"
cap = cv2.VideoCapture(url)

fps = cap.get(cv2.CAP_PROP_FPS)

In [6]:
cv2.namedWindow('camera_screenshot', cv2.WINDOW_AUTOSIZE)
color = (255, 255, 255)

while True:
    camera, frame = cap.read()
    
    if not camera:
        print("Can't read camera")
        break
        
    frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    h = (frame.shape[0] - frame.shape[1]) // 2
    resize_frame = cv2.resize(frame[h:-h, :], dsize=(0, 0), fx=0.5, fy=0.5)    
    
    if int(cap.get(1) % 5 == 0):
        cv2.imwrite('face.jpg', resize_frame)
        
        img = cv2.imread('face.jpg')
        height, width, channels = img.shape
        blob = cv2.dnn.blobFromImage(img, 0.00392, (250, 250), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        
        class_ids = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    class_ids.append(class_id)

        if class_ids.count(0) >= 1:
            print('Helmet detected!!')
            break
        else:
            cv2.putText(resize_frame, 'Please wear a helmet..', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.imshow("camera_screenshot", resize_frame)
    
    cv2.putText(resize_frame, 'Please wear a helmet..', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("camera_screenshot", resize_frame)
    
    # 강제종료 q 키
    if cv2.waitKey(1) == ord('q'):
        break
        
        
cv2.destroyAllWindows()

Helmet detected!!
